In [1]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [2]:
df = pd.read_csv("hotel_reviews.csv")

# drop NaN values for lat and long
df = df.dropna()

# append the positive and negative text reviews
df["combine_review"] = df["Negative_Review"] + df["Positive_Review"]

df["bad_review"] = df["Reviewer_Score"].apply(lambda x: 1 if x < 5 else 0)

df = df[["combine_review", "bad_review"]]

# remove "no negative" + "no positive" from combined review
df["combine_review"] = df["combine_review"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", ""))

In [3]:
df = df.sample(frac = 0.3, replace = False, random_state=42)
df

,combine_review,bad_review
188009,everything was fine We stayed one night over...,0
63099,Breakfast at 25 euro each was a bit steep Ev...,0
27034,They completely missed doing room service on ...,0
206553,Internet very very slow and most of the time ...,0
328796,Maybe a small bar area where we could grab a ...,0
...,...,...
203115,small room lounge closed early good location ...,0
462838,You get cold water when other rooms use hot w...,0
497348,The noice from other rooms was so great and t...,0
16668,Fantastic hotel stayed here due to the distan...,0


In [4]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
docs_raw = df.combine_review

/home/munwon/ENTER/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)

/home/munwon/ENTER/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/munwon/ENTER/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1805: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn("Only {} 'dtype' should be used. {} 'dtype' will "


In [7]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tf.fit(dtm_tf)

# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

/home/munwon/ENTER/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=20, random_state=0)

In [8]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/home/munwon/ENTER/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17    -0.190342  0.011720       1        1  8.436383
19     0.241862  0.143185       2        1  7.698697
8     -0.195126  0.176190       3        1  6.612714
14     0.105482 -0.090482       4        1  6.609811
16    -0.021084  0.194090       5        1  6.033065
9     -0.130355  0.060851       6        1  5.465315
18    -0.139547 -0.041552       7        1  5.136663
10    -0.042672 -0.195200       8        1  5.096161
2     -0.197457  0.157552       9        1  4.856236
4     -0.128333 -0.149728      10        1  4.615040
13    -0.058256 -0.126692      11        1  4.560702
12     0.241490  0.053072      12        1  4.507374
1     -0.058279  0.077687      13        1  4.366691
0      0.126502  0.042853      14        1  4.286312
5      0.067805 -0.040911      15        1  4.121018
11    -0.020974 -0.224667      16        1  3.992234
7      0.131414  0.041821      17        1  3.952913
6      0.093680  0.091398      18        1  3.919271
3      0.089422 -0.088838      19        1  3.512455
15     0.084768 -0.092350      20        1  2.220947, topic_info=           Term          Freq         Total Category  logprob  loglift
2909       good  38562.000000  38562.000000  Default  30.0000  30.0000
6168      small  17974.000000  17974.000000  Default  29.0000  29.0000
811   breakfast  41908.000000  41908.000000  Default  28.0000  28.0000
602         bed  23738.000000  23738.000000  Default  27.0000  27.0000
5683       room  93408.000000  93408.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
6372      staff    907.171756  68423.605078  Topic20  -4.0970  -0.5159
5685      rooms    622.158558  21884.948337  Topic20  -4.4741   0.2469
2909       good    582.748392  38562.454366  Topic20  -4.5395  -0.3850
3791       lift    284.599054   2040.689457  Topic20  -5.2562   1.8373
1232      clean    300.870506  21976.823109  Topic20  -5.2006  -0.4838

[1549 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
12        1  0.033643  absolutely
12        2  0.076297  absolutely
12        4  0.003605  absolutely
12        5  0.771979  absolutely
12        6  0.001202  absolutely
...     ...       ...         ...
7581     20  0.035299       wrong
7586     18  0.930077         xxx
7597     11  0.986341     yoghurt
7612     15  0.907943      zipped
7616      6  0.973935        zuid

[7061 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 20, 9, 15, 17, 10, 19, 11, 3, 5, 14, 13, 2, 1, 6, 12, 8, 7, 4, 16])

In [ ]:
# Topic 1: Hotel Service  vs Topic 14: Hotel Price
# Topic 2: Hotel Bookings
# Topic 3: Hotel Location, Nearby Transportation  vs Topic 9: Transportation
# Topic 4, 19, 20 : Poor Service(noise, wifi, )
